In [1]:
import math
from torch.utils.data import DataLoader
from src.IAMDataset import IAMDataset
from src.model import FullPageHTR
from src.train import ModelTrainer
from src.LabelParser import LabelParser
import torch
from copy import copy

In [2]:
ds = IAMDataset(base_dir="/home/tefan/projects/BachModels/data/raw", embedding_loader=None, sample_set="train")
ds_train, ds_val = torch.utils.data.random_split(ds, [math.ceil(0.8 * len(ds)), math.floor(0.2 * len(ds))])

ds_val.data = copy(ds)
ds_val.data.set_transform_pipeline("val")

In [3]:
from functools import partial

batch_size = 4
pad_tkn_idx, eos_tkn_idx = ds.embedding_loader.encode_labels(["<PAD>", "<EOS>"])
collate_fn = partial(
        IAMDataset.collate_fn, pad_val=pad_tkn_idx, eos_tkn_idx=eos_tkn_idx
)
num_workers = 4
dl_train = DataLoader(
    ds_train,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True,
)
dl_val = DataLoader(
    ds_val,
    batch_size=2 * batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True,
)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FullPageHTR(ds.embedding_loader).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
trainer = ModelTrainer("Testing_run", model, ds_name="IAM_forms" ,train_data=dl_train, val_data=dl_val, optimizer=optimizer, num_epochs=10, device=device)

trainer.train()

/home/tefan/projects/BachModels/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tefan/projects/BachModels/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: stefannastasa. Use `wandb login --relogin` to force relogin


  0%|          | 0/10 [00:01<?, ?it/s]


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 1 but got size 392 for tensor number 1 in the list.